In [1]:
import os
import pandas as pd
import numpy as np
import urllib.request, json
from shapely.geometry import Polygon

In [30]:
def get_area(filename, geojson_prop):
    
    with open(os.path.join('..','..','data','shapefiles',filename), 'r') as file:
        data_polygon = json.load(file)

    df_area = []
    for feature in data_polygon['features']:
        poly = Polygon(feature['geometry']['coordinates'][0])

        prop = feature['properties']

        province, district, subdist = prop[geojson_prop['province']], prop[geojson_prop['district']], prop[geojson_prop['subdist']]

        df_area.append([province, district, subdist, round(poly.area*111111,2)])

    df_area = pd.DataFrame.from_records(df_area)
    df_area.columns = ['province','district', 'subdist', 'area']
    
    df_area.dropna(axis='index', how='any', inplace=True)
    df_area.drop_duplicates(keep='first', inplace=True)
    
    return df_area

In [31]:
df_area_BK = get_area(
    filename =  'Bangkok-subdistricts.geojson',
    geojson_prop = {
        'province':'PNAME',
        'district':'DNAME',
        'subdist':'SNAME'
    }
)
df_area_NK = get_area(
    filename =  'Nakhon-subdistricts.geojson',
    geojson_prop = {
        'province':'PV_TN',
        'district':'AP_TN',
        'subdist':'TB_TN'
    }
)
df_area_KB = get_area(
    filename =  'Krabi-subdistricts.geojson',
    geojson_prop = {
        'province':'PV_TN',
        'district':'AP_TN',
        'subdist':'TB_TN'
    }
)

print('Bangkok:', len(df_area_BK))
print('Nakhon:', len(df_area_NK))
print('Krabi:', len(df_area_KB))

Bangkok: 169
Nakhon: 170
Krabi: 65


In [39]:
df_area = pd.concat([df_area_BK,df_area_NK,df_area_KB])
df_area.reset_index(drop=True, inplace=True)
df_area.head()

,province,district,subdist,area
0,กรุงเทพมหานคร,เขตบางเขน,แขวงท่าแร้ง,212.97
1,กรุงเทพมหานคร,เขตคลองสามวา,แขวงทรายกองดินใต้,161.16
2,กรุงเทพมหานคร,เขตคลองสามวา,แขวงทรายกองดิน,98.90
3,กรุงเทพมหานคร,เขตราชเทวี,แขวงมักกะสัน,22.99
4,กรุงเทพมหานคร,เขตพระนคร,แขวงตลาดยอด,1.75


In [40]:
df_area.tail()

,province,district,subdist,area
399,กระบี่,เกาะลันตา,เกาะลันตาใหญ่,2.31
400,กระบี่,เกาะลันตา,เกาะลันตาใหญ่,31.58
401,กระบี่,ปลายพระยา,เขาต่อ,965.90
402,กระบี่,ปลายพระยา,ปลายพระยา,2161.03
403,กระบี่,ปลายพระยา,เขาเขน,1012.55


In [42]:
df_area.to_csv(os.path.join('..','..','data','shapefiles','area.csv'), index=None)